In [1]:
#pip install easydl


the cSCC dataset is 10x visium technology; the HER2+ dataset ST w/ 100 micrometer spots.

thus, it is better to test with cSCC data

In [2]:
import torch
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as tf
from tqdm import tqdm
from predict import *

#in HIST2ST, collections.Iterable is imported via 
# from HIST2ST import * --> from transformer import* 
# --> from easydl import * --> from .pytorch import *
# from .utils import * --> from collections import Iterable.
# the alias Iterable has been deprecated however,
# so you need a workaround: second answer: https://stackoverflow.com/questions/72032032/importerror-cannot-import-name-iterable-from-collections-in-python
import collections.abc
collections.Iterable = collections.abc.Iterable


from HIST2ST import *
from dataset import ViT_HER2ST, ViT_SKIN
from scipy.stats import pearsonr,spearmanr
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from copy import deepcopy as dcp
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score

[easydl] tensorflow not available!


# Data Loading

In [3]:
name=[*[f'A{i}' for i in range(2,7)],*[f'B{i}' for i in range(1,7)],
      *[f'C{i}' for i in range(1,7)],*[f'D{i}' for i in range(1,7)],
      *[f'E{i}' for i in range(1,4)],*[f'F{i}' for i in range(1,4)],*[f'G{i}' for i in range(1,4)]]
patients = ['P2', 'P5', 'P9', 'P10']
reps = ['rep1', 'rep2', 'rep3']
skinname = []
for i in patients:
    for j in reps:
        skinname.append(i+'_ST_'+j)
device='cuda'
tag='5-7-2-8-4-16-32'
k,p,d1,d2,d3,h,c=map(lambda x:int(x),tag.split('-'))
dropout=0.2
random.seed(12000)
np.random.seed(12000)
torch.manual_seed(12000)
torch.cuda.manual_seed(12000)
torch.cuda.manual_seed_all(12000)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


# Hist2ST Prediction

### To run the trained model, please select the trained model and replace the value of the variable fold with the number in the name of the selected trained model.

In [4]:
fold=5
data='cscc'
prune='Grid' if data=='her2st' else 'NA'
genes=171 if data=='cscc' else 785

the model was trained for the 785 HVGs in HER2+ breast cancer samples. thus, if we use default models, we will predict genes that will most likely not have anything to do with the liver. therefore, it does not make sense to use default models in the first step of the thesis.

the selection for the genes has already been made before starting to train as only the selected genes will be normalized and put into self.exp_dict of the dataset

In [5]:
#load dataset with the ViT_HER2ST function if data == "her2st"
#fold is the fold for the LOOCV. 
#note that the first section A1 was excluded, so fold 5 will be B1 instead of A6
testset = pk_load(fold,'test',dataset=data,flatten=False,adj=True,ori=True,prune=prune)

['P5_ST_rep3']
Loading imgs...
Loading metadata...


"For evaluating the gene expression
prediction accuracy, we conducted leave-one-out crossvalidation. Specifically, for each section, we used the left
sections to train our model and predict gene expression
for the section. All predictions were collected to assess
the model performance"

In [6]:
test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)

the next two lines had to be commented out for the model to run on the cSCC dataset

In [7]:
#label=testset.label[testset.names[0]]
#genes=785

In [8]:
model=Hist2ST(
    depth1=d1, depth2=d2,depth3=d3,n_genes=genes, 
    kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5, 
)

n_pos 64
dim 1024


In [9]:
#model.load_state_dict(torch.load(f'./model/{fold}-Hist2ST.ckpt'))

In [10]:
model.load_state_dict(torch.load(f'./model/{fold}_skin-Hist2ST.ckpt'))

<All keys matched successfully>

internally, the test function runs pred = model(patch, position, adj)[0] the following data from the test_loader

In [11]:
for patch, position, exp, adj, *_, center in test_loader:
    print(patch.shape, position.shape, exp.shape, adj.shape, center.shape)

torch.Size([1, 521, 3, 112, 112]) torch.Size([1, 521, 2]) torch.Size([1, 521, 171]) torch.Size([1, 521, 521]) torch.Size([1, 521, 2])


In [12]:
position

tensor([[[14, 32],
         [14, 36],
         [15, 29],
         ...,
         [42, 36],
         [42, 38],
         [42, 40]]])

In [13]:
pred, gt = test(model, test_loader,'cuda')

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

patches shape torch.Size([1, 521, 3, 112, 112])
centers shape torch.Size([1, 521, 2])
centers max 42
centers min 14


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.32s/it]


the following cell shows what data is used inside the test function.
I dont get what the difference between center and position is yet though

In [13]:
for patch, position,exp, adj, *_, center in test_loader:
    print(f"patch: {patch.shape}")
    print(f"position: {position.shape} are the x,y centers for the forward pass. ")
    print(f"Expression: {exp.shape}")
    print(f"adj = adjacency matrix {adj.shape}")
    print(f"center: {center.shape}")

patch: torch.Size([1, 521, 3, 112, 112])
position: torch.Size([1, 521, 2]) are the x,y centers for the forward pass. 
Expression: torch.Size([1, 521, 171])
adj = adjacency matrix torch.Size([1, 521, 521])
center: torch.Size([1, 521, 2])


In [14]:
pred, gt


(AnnData object with n_obs × n_vars = 521 × 171
     obsm: 'spatial',
 AnnData object with n_obs × n_vars = 521 × 171
     obsm: 'spatial')

521 spots (?) x 171 genes

In [15]:
pred.obsm["spatial"].shape

(521, 2)

In [16]:
# spatial are the center (ct) locations of the spots
pred.obsm["spatial"]

array([[4581., 8002.],
       [4586., 8809.],
       [4766., 7408.],
       ...,
       [9634., 8815.],
       [9628., 9224.],
       [9623., 9620.]], dtype=float32)

In [17]:
pred.X.shape

(521, 171)

In [18]:
#get_R??

In [19]:
#returns tuple of all array(r values), array(all pvalues)
len(get_R(pred,gt))

2

In [20]:
R=get_R(pred,gt)[0]
print('Pearson Correlation:',np.nanmean(R))


# clus,ARI=cluster(pred,label)
# print('ARI:',ARI)


Pearson Correlation: 0.10740173624584474
